In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import us

In [2]:
amz = pd.read_csv("Data/CSV//amazon_labelled_locations.csv")
moodys = pd.read_csv("Data/CSV//MoodysData-ACS.csv")

In [3]:
#find  rows from amazon that have no county
amz[amz["County"].isnull()]

,State,Code,Location,Square Feet,Year Opened,Description of Operation,Type,County,City,Year
143,Maryland,BWIO/BWI2,"2010 Broening Highway, Baltimore, Maryland, US...",1017600.0,March 2015,Small Sortable,Fulfillment,NaN,Baltimore,2015.0
240,Maryland,UMD1,"5001 Holabird Ave., Baltimore, Maryland, USA, ...",NaN,March 2015,Prime Now Hub for Baltimore Co-located in BWIS,Now,NaN,Baltimore,2015.0
262,Virginia,UVA2,"804 Greenbrier Circle, Chesapeake, Virginia, U...",18300.0,February 2016,Prime Now Hub for Chesapeake VA,Now,NaN,Chesapeake,2016.0
288,Maryland,BWI5/UMD1,"5001 Holabird Ave., Baltimore, Maryland, USA, ...",345000.0,October 2014,Sortation Center connected by conveyor to the ...,Sortation,NaN,Baltimore,2014.0


In [4]:
#ditch rows from amazon that have no county
amz_dropped =amz.dropna(subset=["County"])

In [5]:
#rename the unnamed columsn from moodys
cols = list(moodys.columns)
#print (cols)
cols[0]="county"
cols[1]="moodys_county"
#print (cols)
moodys.columns= cols

In [6]:
#Amazon has full state names, add a column of State abbreviations eg, CA, vs California
stateMap =us.states.mapping('name','abbr')
def stateCode(state):
    return stateMap[state.title()]

In [7]:
amz_dropped["ST"]=np.vectorize(stateCode)(amz_dropped["State"])

C:\Users\akwes\Anaconda2\envs\cappy\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [8]:
#HBD
#We want to string match the county names between the two datasets
#Make a new amazon column "County, ST" the space is important
amz_dropped["moodys_county"]=amz_dropped["County"]+", "+amz_dropped["ST"]

C:\Users\akwes\Anaconda2\envs\cappy\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
amz_dropped.head()

,State,Code,Location,Square Feet,Year Opened,Description of Operation,Type,County,City,Year,ST,moodys_county
0,Arizona,DPX1,"500 S. 48th Street, Phoenix, Arizona, USA, 85034",NaN,May 2015,Delivery Station for Phoenix West Valley Co-lo...,Delivery Station,Maricopa County,Phoenix,2015.0,AZ,"Maricopa County, AZ"
1,Arizona,DPX3,"550 E. Elliot Rd, Chandler . Arizona, USA, 85225",91200.0,May 2017,Delivery Station for Phoenix East Valley,Delivery Station,Maricopa County,Chandler,2017.0,AZ,"Maricopa County, AZ"
2,California,DSF1,"990 Beecher St, San Leandro, California, USA, ...",42400.0,September 2013,Delivery Station for Oakland,Delivery Station,Alameda County,San Leandro,2013.0,CA,"Alameda County, CA"
3,California,DSF2,"Candlestick Park Delivery Station, San Francis...",NaN,September 2013,Delivery Station for Bay Area,Delivery Station,San Francisco County,San Francisco,2013.0,CA,"San Francisco County, CA"
4,California,DSF3,"1700 Montague Expy, San Jose, California, USA,...",25800.0,2013,Delivery Station for San Jose,Delivery Station,Santa Clara County,San Jose,2013.0,CA,"Santa Clara County, CA"


In [10]:
#no left join amazon on the moodys
merged=pd.merge(amz_dropped,moodys,how="left",on="moodys_county")

In [11]:
#What counties are missing from the moodys data
print(merged[merged["State_y"].isnull()]["County"].unique())

['San Francisco County' 'New York County' 'Denver County' 'Hartford County'
 'New Haven County' 'Jefferson County' 'Kings County' 'Davidson County']


In [12]:
#clean up merged dataframe

#rename some stuff and delete some stuff
c1 = merged.drop(axis=1,labels=["Year Opened","County","ST","State_x","Include in Amazon Dataset","State_y"])
c2 = c1.rename(columns={"Code_x":"AMZ_Code","Code_y":"moodys_code","Years":"ACS_year",})


In [13]:
with pd.option_context('display.max_columns', 50,):
    display(c2.head())


,AMZ_Code,Location,Square Feet,Description of Operation,Type,City,Year,moodys_county,county,moodys_code,ACS_year,Current Senior Most Rating*,Tax Backed Rating Description,Population (ACS Data),Per Capita Income (ACS Data),Median Family Income (ACS Data),Median Home Value (ACS Data),Median Gross Rent (ACS Data),Occupied Housing Units (ACS Data),Seasonal Homes (ACS Data),No. Persons/Household (ACS Data),Median Age (ACS Data),% Below Poverty Level (ACS Data)
0,DPX1,"500 S. 48th Street, Phoenix, Arizona, USA, 85034",NaN,Delivery Station for Phoenix West Valley Co-lo...,Delivery Station,Phoenix,2015.0,"Maricopa County, AZ","Maricopa County, AZ",160.0,2017.0,Aaa,ISSUER LT RATING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DPX1,"500 S. 48th Street, Phoenix, Arizona, USA, 85034",NaN,Delivery Station for Phoenix West Valley Co-lo...,Delivery Station,Phoenix,2015.0,"Maricopa County, AZ",NaN,160.0,2016.0,Aaa,ISSUER LT RATING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DPX1,"500 S. 48th Street, Phoenix, Arizona, USA, 85034",NaN,Delivery Station for Phoenix West Valley Co-lo...,Delivery Station,Phoenix,2015.0,"Maricopa County, AZ",NaN,160.0,2015.0,Aaa,ISSUER LT RATING,"4,018,143","27,832","64,751","187,100",962,"1,442,518","81,978",3.0,36.0,17.0
3,DPX1,"500 S. 48th Street, Phoenix, Arizona, USA, 85034",NaN,Delivery Station for Phoenix West Valley Co-lo...,Delivery Station,Phoenix,2015.0,"Maricopa County, AZ",NaN,160.0,2014.0,Aaa,ISSUER LT RATING,"3,947,382","27,477","64,072","175,600",952,"1,424,244","78,934",3.0,35.0,17.0
4,DPX1,"500 S. 48th Street, Phoenix, Arizona, USA, 85034",NaN,Delivery Station for Phoenix West Valley Co-lo...,Delivery Station,Phoenix,2015.0,"Maricopa County, AZ",NaN,160.0,2013.0,Aaa,ISSUER LT RATING,"3,889,161","27,256","63,910","176,500",943,"1,411,727","73,195",3.0,35.0,17.0


In [14]:
#write out the merged dataframe to a new csv
c2.to_csv("Data/CSV//amazon_plus_moodys-ACS.csv",index=False)